In [ ]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
%run "/Formula1/includes/configuration"

In [ ]:
file_path = f'{raw_folder_path}/pit_stops.json'

dbutils.fs.head(file_path)

[Truncated to first 65536 bytes]
Out[61]: '[\r\n   {\r\n      "raceId":841,\r\n      "driverId":153,\r\n      "stop":1,\r\n      "lap":1,\r\n      "time":"17:05:23",\r\n      "duration":26.898,\r\n      "milliseconds":26898\r\n   },\r\n   {\r\n      "raceId":841,\r\n      "driverId":30,\r\n      "stop":1,\r\n      "lap":1,\r\n      "time":"17:05:52",\r\n      "duration":25.021,\r\n      "milliseconds":25021\r\n   },\r\n   {\r\n      "raceId":841,\r\n      "driverId":17,\r\n      "stop":1,\r\n      "lap":11,\r\n      "time":"17:20:48",\r\n      "duration":23.426,\r\n      "milliseconds":23426\r\n   },\r\n   {\r\n      "raceId":841,\r\n      "driverId":4,\r\n      "stop":1,\r\n      "lap":12,\r\n      "time":"17:22:34",\r\n      "duration":23.251,\r\n      "milliseconds":23251\r\n   },\r\n   {\r\n      "raceId":841,\r\n      "driverId":13,\r\n      "stop":1,\r\n      "lap":13,\r\n      "time":"17:24:10",\r\n      "duration":23.842,\r\n      "milliseconds":23842\r\n   },\r\n   {\r\n      

whenever you have a multiline json file make sure to add .option('multiline', True) property.

In [ ]:
from pyspark.sql.functions import col


schema = "raceId int, driverId int, stop String, lap int, time String, duration String, milliseconds int"
df = spark.read.format('json').schema(schema).option('multiLine',True).load(file_path)
df.show(3)


+------+--------+----+---+--------+--------+------------+
|raceId|driverId|stop|lap|    time|duration|milliseconds|
+------+--------+----+---+--------+--------+------------+
|   841|     153|   1|  1|17:05:23|  26.898|       26898|
|   841|      30|   1|  1|17:05:52|  25.021|       25021|
|   841|      17|   1| 11|17:20:48|  23.426|       23426|
+------+--------+----+---+--------+--------+------------+
only showing top 3 rows



In [ ]:
df.printSchema()

root
 |-- raceId: integer (nullable = true)
 |-- driverId: integer (nullable = true)
 |-- stop: string (nullable = true)
 |-- lap: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- milliseconds: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import current_timestamp
new_df = df.withColumn('ingestion_date', current_timestamp())\
            .withColumnRenamed("race_id","race_id") \
            .withColumnRenamed("driverId", "driver_id")

new_df.show(4)

+------+---------+----+---+--------+--------+------------+--------------------+
|raceId|driver_id|stop|lap|    time|duration|milliseconds|      ingestion_date|
+------+---------+----+---+--------+--------+------------+--------------------+
|   841|      153|   1|  1|17:05:23|  26.898|       26898|2023-06-16 14:22:...|
|   841|       30|   1|  1|17:05:52|  25.021|       25021|2023-06-16 14:22:...|
|   841|       17|   1| 11|17:20:48|  23.426|       23426|2023-06-16 14:22:...|
|   841|        4|   1| 12|17:22:34|  23.251|       23251|2023-06-16 14:22:...|
+------+---------+----+---+--------+--------+------------+--------------------+
only showing top 4 rows



In [ ]:
dbutils.fs.ls(processed_folder_path)

Out[65]: [FileInfo(path='dbfs:/mnt/formularacedata/processed/circuits/', name='circuits/', size=0, modificationTime=1686657879000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/constructors/', name='constructors/', size=0, modificationTime=1686899550000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/driver/', name='driver/', size=0, modificationTime=1686917781000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/pit_stops/', name='pit_stops/', size=0, modificationTime=1686925132000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/races/', name='races/', size=0, modificationTime=1686741009000)]

In [ ]:
new_df.show(4)

+------+---------+----+---+--------+--------+------------+--------------------+
|raceId|driver_id|stop|lap|    time|duration|milliseconds|      ingestion_date|
+------+---------+----+---+--------+--------+------------+--------------------+
|   841|      153|   1|  1|17:05:23|  26.898|       26898|2023-06-16 14:22:...|
|   841|       30|   1|  1|17:05:52|  25.021|       25021|2023-06-16 14:22:...|
|   841|       17|   1| 11|17:20:48|  23.426|       23426|2023-06-16 14:22:...|
|   841|        4|   1| 12|17:22:34|  23.251|       23251|2023-06-16 14:22:...|
+------+---------+----+---+--------+--------+------------+--------------------+
only showing top 4 rows



In [ ]:
dest_path = f"{processed_folder_path}/pit_stops"
new_df.write.mode("overwrite").format('parquet').option('path',dest_path).save()

In [ ]:
spark.read.parquet(dest_path).show(3)

+------+---------+----+---+--------+--------+------------+--------------------+
|raceId|driver_id|stop|lap|    time|duration|milliseconds|      ingestion_date|
+------+---------+----+---+--------+--------+------------+--------------------+
|   841|      153|   1|  1|17:05:23|  26.898|       26898|2023-06-16 14:22:...|
|   841|       30|   1|  1|17:05:52|  25.021|       25021|2023-06-16 14:22:...|
|   841|       17|   1| 11|17:20:48|  23.426|       23426|2023-06-16 14:22:...|
+------+---------+----+---+--------+--------+------------+--------------------+
only showing top 3 rows



In [ ]:
dbutils.notebook.exit("success")